<p style="color:#FFF; background:#951587; padding:12px; font-size:20px; font-style:italic; text-align:center">
<span style="width:100%; display:inline-block; text-align:left">ABARKAN Suhaïla</span>
<span style="font-size:48px; font-style:normal"><b>Conte de l'âne triste (V2)</b></span><br> 
<span style="width:49%; display:inline-block; text-align:left">06/06/2024 - 12/07/2024</span>
<span style="width:49%; display:inline-block; text-align:right">CMI ISI</span></p>

---
<h2 style="padding:16px; color:#FFF; background:#951587">Imports nécessaires</h2>

In [17]:
import pandas as pd
import re
import os
import requests

---
<h2 style="padding:16px; color:#FFF; background:#951587">Création de la base de données sous un fichier .csv à l'aide de fichiers Excel</h2>

In [18]:
def list_files(directory):
    """Récupération de tous les fichiers excel (.xls et .xlsx) dans le dossier "directory"."""
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.xls', '.xlsx'))]

`Indication :` Pour le bon déroulement de la fonction suivante appelée **extract_info**, il faudra impérativement conserver la même façon de nommer tous les fichiers excel de l'enquête (les fichiers dans le dossier "reponses").

In [19]:
def extract_info(file):
    """Extraction du numéro INSEE et de l'identifiant du locuteur dans le nom des fichiers Excel."""
    match = re.match(r'PaRL_(\d{5})_(e\d{2})_Questionnaire_Conte', file)
    if match:
        return match.groups()
    return None, None  # dans le cas où les infos sont pas/mal renseignées

`Indication :` Pour le bon déroulement de la fonction suivante appelée **filter_excel**, il faudra impérativement conserver la même façon de remplir tous les fichiers excel de l'enquête (les fichiers dans le dossier "reponses").

In [20]:
def filter_excel(file):
    """Filtrage des fichiers Excel pour récupérer les informations du conte et du locuteur."""
    df = pd.read_excel(file, sheet_name=0)
    # lignes du conte commençant par des numéros (1, 2, ..., 44)
    conte_df = df[df.iloc[:, 0].astype(str).str.match(r'^\d+.*')]
    # lignes contenant les infos du locuteur
    locuteur_df = df.iloc[-8:, 2].tolist()
    return conte_df, locuteur_df

In [21]:
def get_geo_info(code_insee):
    """Récupération des informations géographiques selon le code INSEE, via une api."""
    url = f'https://geo.api.gouv.fr/communes/{code_insee}?fields=nom,code,codesPostaux,surface,population,centre,contour,departement'
    response = requests.get(url)
    data = response.json()
    return {
        'ville': data['nom'],
        'departement': data['departement']['nom'],
        'x': data['centre']['coordinates'][0],
        'y': data['centre']['coordinates'][1]
    }

`Indication` : Dé-commentez la fonction **split_phrase**, ainsi que deux lignes en commentaires dans **create_csv**
si vous souhaitez un découpage par mots pour chaque phrase dans la création de ce fichier csv.

In [22]:
#def split_phrase(phrase):
#   """Séparation mot à mot d'une phrase."""
#   mots = re.split(r"[^\w']+", phrase.replace(',', '').replace("'", ' '))
#   return {f'mot_{i+1}': mot for i, mot in enumerate(mots)}

In [23]:
def create_csv(files, output_csv='bdd_conte_v2.csv'):
    """Création de fichier .csv sous 'bdd_conte_v2.csv' en remplissant avec les informations des fichier Excel."""
    columns = ['code_INSEE', 'ville', 'departement', 'x', 'y', 'identifiant', 'age', 'lieu_de_naissance',
               'sexe', 'lieu_de_residence', 'parler_local', 'commune_apprentissage', 'profession',
               'adresse_courriel', 'num_phrase', 'phrase', 'commentaire']
    
    df_output = pd.DataFrame(columns=columns)

    for file in files:
        df, locuteur_info = filter_excel(file)
        code_insee, identifiant = extract_info(file.split('/')[-1])
        geo_info = get_geo_info(code_insee)
        new_rows = []
        for _, row in df.iterrows():
            num_phrase = row.iloc[0]
            phrase = row.iloc[2]
            #mots = split_phrase(phrase)
            new_row = {
                'code_INSEE': code_insee,
                'ville': geo_info['ville'],
                'departement': geo_info['departement'],
                'x': geo_info['x'],
                'y': geo_info['y'],
                'identifiant': identifiant,
                'age': locuteur_info[0],
                'lieu_de_naissance': locuteur_info[1],
                'sexe': locuteur_info[2],
                'lieu_de_residence': locuteur_info[3],
                'parler_local': locuteur_info[4],
                'commune_apprentissage': locuteur_info[5],
                'profession': locuteur_info[6],
                'adresse_courriel': locuteur_info[7],
                'num_phrase': num_phrase,
                'phrase': phrase,
                'commentaire': row.iloc[3],
                #**mots
            }
            new_rows.append(new_row)
        df_output = pd.concat(
            [df_output, pd.DataFrame(new_rows)], ignore_index=True)

    df_output.to_csv(output_csv, index=False)
    print(f"Base de données créée et enregistrée sous {output_csv}")

<h4 style="padding:16px; color:#FFF; background:#C180BA">Création du fichier .csv</h4>

`Indications` : Si le dossier "reponses" n'existe pas, merci de le créer et de mettre toutes les réponses de l'enquête (sous format excel) à l'intérieur, afin de faciliter l'accés à ces fichiers pour la création de la base de données. Ce dossier doit se trouver au même endroit que ce notebook.

In [24]:
directory = 'reponses'
files = list_files(directory)
create_csv(files)

/var/folders/90/ydgs6yxx0h3d38dgcdtcshsw0000gn/T/ipykernel_3107/460545148.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_output = pd.concat(


Base de données créée et enregistrée sous bdd_conte_v2.csv
